In [ ]:
# Clone repository
!git clone https://github.com/Tushar365/finetune-changeformer.git
%cd finetune-changeformer

Cloning into 'finetune-changeformer'...
remote: Enumerating objects: 1308, done.
remote: Counting objects: 100% (1308/1308), done.
remote: Compressing objects: 100% (569/569), done.
remote: Total 1308 (delta 736), reused 1297 (delta 727), pack-reused 0 (from 0)
Receiving objects: 100% (1308/1308), 14.03 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (736/736), done.
/content/finetune-changeformer


Those three Colab lines batch-edit every .py file under /content/finetune-changeformer to add weights_only=False to torch.load calls so legacy ChangeFormer checkpoints load correctly on newer PyTorch builds that otherwise expect “weights-only” safe loading by default or emit warnings/errors.​

Line 1: Rewrites torch.load(self.args.pretrain) to torch.load(self.args.pretrain, weights_only=False) to allow loading full pickled objects from a trusted pretrain path.​

Line 2: Injects weights_only=False into torch.load(os.path.join(self.checkpoint_dir, checkpoint_name), …) without removing existing args, preserving call behavior while restoring permissive deserialization.​

Line 3: Updates torch.load(ckpt_path) to torch.load(ckpt_path, weights_only=False) for generic checkpoint paths.​

Why needed: Recent PyTorch versions promote or default to safer “weights-only” deserialization, which can break older repos that saved full objects; explicitly setting weights_only=False restores compatibility but should be used only for trusted checkpoints due to pickle execution risks.​

Tip: Prefer weights_only=True plus model.load_state_dict(...) when loading untrusted files, aligning with PyTorch’s recommended best practice.

In [ ]:
!find /content/finetune-changeformer -type f -name "*.py" -exec sed -i 's|torch.load(self.args.pretrain)|torch.load(self.args.pretrain, weights_only=False)|g' {} +
!find /content/finetune-changeformer -type f -name "*.py" -exec sed -i 's|torch.load(os.path.join(self.checkpoint_dir, checkpoint_name),|torch.load(os.path.join(self.checkpoint_dir, checkpoint_name), weights_only=False,|g' {} +
!find /content/finetune-changeformer -type f -name "*.py" -exec sed -i 's|torch.load(ckpt_path)|torch.load(ckpt_path, weights_only=False)|g' {} +


In [ ]:
!pip install torch torchvision einops matplotlib timm tqdm scipy gdown


In [ ]:
# download the dataser --- LEVIR-CD256.zip
!gdown 1NLD2JptwxXxKj9kjSdfJ2abhsKkVK_mk

Downloading...
From (original): https://drive.google.com/uc?id=1NLD2JptwxXxKj9kjSdfJ2abhsKkVK_mk
From (redirected): https://drive.google.com/uc?id=1NLD2JptwxXxKj9kjSdfJ2abhsKkVK_mk&confirm=t&uuid=c44a4450-4ab0-4112-bd5b-aa25959a5e5f
To: /content/finetune-changeformer/LEVIR-CD256.zip
100% 2.48G/2.48G [00:31<00:00, 78.0MB/s]


In [ ]:
#unzip data
!unzip LEVIR-CD256.zip


In [ ]:
# Download pretrained model for LEVIR-CD (from the README link)
!wget https://github.com/wgcban/ChangeFormer/releases/download/v0.1.0/CD_ChangeFormerV6_LEVIR_b16_lr0.0001_adamw_train_test_200_linear_ce_multi_train_True_multi_infer_False_shuffle_AB_False_embed_dim_256.zip

# Also download initialization weights
!wget https://www.dropbox.com/s/undtrlxiz7bkag5/pretrained_changeformer.pt

# Create checkpoint directories and extract
!mkdir -p checkpoints/ChangeFormer_LEVIR/
!unzip CD_ChangeFormerV6_LEVIR_b16_lr0.0001_adamw_train_test_200_linear_ce_multi_train_True_multi_infer_False_shuffle_AB_False_embed_dim_256.zip -d checkpoints/ChangeFormer_LEVIR/


--2025-10-09 11:01:17--  https://github.com/wgcban/ChangeFormer/releases/download/v0.1.0/CD_ChangeFormerV6_LEVIR_b16_lr0.0001_adamw_train_test_200_linear_ce_multi_train_True_multi_infer_False_shuffle_AB_False_embed_dim_256.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/434410116/d0824525-6878-4456-bc1a-eb4fbed47a4c?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-09T11%3A54%3A04Z&rscd=attachment%3B+filename%3DCD_ChangeFormerV6_LEVIR_b16_lr0.0001_adamw_train_test_200_linear_ce_multi_train_True_multi_infer_False_shuffle_AB_False_embed_dim_256.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-09T10%3A53%3A23Z&ske=2025-10-09T11%3A54%3A04Z&sks=b&skv=2018-11-09&sig=pvWUKlZKRduJakqrqQPFxjZ3hxZ2Gs9HkQol

In [ ]:
#finetune
!python main_cd.py \
    --net_G ChangeFormerV6 \
    --img_size 256 \
    --embed_dim 256 \
    --loss ce \
    --checkpoint_root /content/finetune-changeformer/checkpoints \
    --max_epochs 2 \
    --data_name LEVIR \
    --batch_size 8 \
    --project_name 1_CHANGEFORMER_FINETUNED \
    --pretrain /content/finetune-changeformer/checkpoints/ChangeFormer_LEVIR/CD_ChangeFormerV6_LEVIR_b16_lr0.0001_adamw_train_test_200_linear_ce_multi_train_True_multi_infer_False_shuffle_AB_False_embed_dim_256/best_ckpt.pt

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/content/finetune-changeformer/main_cd.py:8: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  torch.serialization.add_safe_globals([np.core.multiarray.scalar])
True
[0]
initialize network with normal
cuda:0
================ (Thu Oct  9 08:24:01 2025) ================
g

In [ ]:
# Zip the entire checkpoint folder
!zip -r 1_CHANGEFORMER_FINETUNED.zip /content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/

# Download the zip file
from google.colab import files
files.download('1_CHANGEFORMER_FINETUNED.zip')


  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/ (stored 0%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/log_test.txt (deflated 52%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/scores_dict.npy (deflated 38%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/train_acc.npy (deflated 41%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/log.txt (deflated 69%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/0.758170436996565.txt (stored 0%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/last_ckpt.pt (deflated 7%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/val_acc.npy (deflated 41%)
  adding: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/best_ckpt.pt (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#importing model my finetuned model
!gdown 1WxRlCWvmp20Gg6TwtFX33ax90QuQhIm2

Downloading...
From (original): https://drive.google.com/uc?id=1WxRlCWvmp20Gg6TwtFX33ax90QuQhIm2
From (redirected): https://drive.google.com/uc?id=1WxRlCWvmp20Gg6TwtFX33ax90QuQhIm2&confirm=t&uuid=a7d6e432-5f70-4e46-bda9-0590a565a4c6
To: /content/finetune-changeformer/1_CHANGEFORMER_FINETUNED.zip
100% 608M/608M [00:08<00:00, 69.2MB/s]


In [ ]:
!unzip 1_CHANGEFORMER_FINETUNED.zip

Archive:  1_CHANGEFORMER_FINETUNED.zip
   creating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/log_test.txt  
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/scores_dict.npy  
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/train_acc.npy  
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/log.txt  
 extracting: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/0.758170436996565.txt  
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/last_ckpt.pt  
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/val_acc.npy  
  inflating: content/finetune-changeformer/checkpoints/1_CHANGEFORMER_FINETUNED/best_ckpt.pt  
